In [16]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

Load Data

In [10]:
df = pd.read_csv('preprocessed_bird_data.csv')
df = df.drop(['Class', 'Units', 'All_countries', 'Location', 'Country'], axis=1)
df

,ID,Binomial,1950,1951,1952,1953,1954,1955,1956,1957,...,2013,2014,2015,2016,2017,2018,2019,2020,Native,Unnamed: 102
0,900000,Amazilia_yucatanensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.065862,0.063408,0.062666,0.061329,0.062801,0.060782,0.059463,0.059019,1,NaN
1,900001,Ammospiza_caudacuta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.030482,0.030065,0.030092,0.030277,0.030604,0.031654,0.032999,0.032615,1,NaN
2,900002,Buteo_albonotatus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.074429,0.088306,0.076481,0.078282,0.088370,0.083504,0.099746,0.099536,1,NaN
3,900003,Contopus_pertinax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.502271,0.492260,0.491091,0.493591,0.493640,0.490580,0.479314,0.470276,1,NaN
4,900004,Cynanthus_latirostris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.293427,0.292854,0.292460,0.292380,0.291198,0.290369,0.289918,0.289687,1,NaN
5,900005,Leuconotopicus_arizonae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.225670,0.223792,0.222002,0.216864,0.215200,0.213837,0.212822,0.213893,1,NaN
6,900006,Icterus_gularis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001799,0.001708,0.001665,0.001670,0.001511,0.001417,0.001274,0.001201,1,NaN
7,900007,Icterus_pectoralis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005088,0.004290,0.003642,0.003268,0.003053,0.002713,0.002510,0.002340,1,NaN
8,900008,Junco_phaeonotus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.363258,38.384300,37.963268,37.399590,36.462292,35.627579,34.407829,33.535912,1,NaN
9,900009,Laterallus_jamaicensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.059170,0.059238,0.059213,0.059327,0.058901,0.058832,0.060137,0.063493,1,NaN


Prepare Regression Data for Conservation Status Predictions

In [ ]:
#features selected for regression trends
#since we are only looking at birds or mammals, we can encode class to be 1 or 0, 1 = mammal, 0 = bird
reg_features = ['class', 'avg2', 'avg5', 'avg10',
                'curr_clusters', 'avg_dist_clusters', 'max_dist_clusters']


X_reg = df[reg_features]
y = df['con_status'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression with class weights
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)
logreg.fit(X_train, y_train)

# Predict & Evaluate
y_pred_logreg = logreg.predict(X_test)
print("=== Logistic Regression ===")
print("Accuracy:", accuracy_score(y_test, y_pred_logreg))
print(classification_report(y_test, y_pred_logreg))

# Train Naive Bayes
nb = GaussianNB()
nb.fit(X_train, y_train)

# Predict & Evaluate
y_pred_nb = nb.predict(X_test)
print("=== Naive Bayes ===")
print("Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

KeyError: "None of [Index(['class', 'avg2', 'avg5', 'avg10', 'curr_clusters', 'avg_dist_clusters',\n       'max_dist_clusters'],\n      dtype='object')] are in the [columns]"